In [2]:
## IMPORTS 

import os
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from ast import literal_eval
import json
from sklearn.cluster import KMeans


In [3]:
## BASE HISTORICA 

# Definir o diretório contendo os arquivos CSV
diretorio = "challenge-webmedia-e-globo-2023/files/treino"

# Listar todos os arquivos CSV na pasta
arquivos_csv = [f for f in os.listdir(diretorio) if f.endswith(".csv")]

# Criar uma lista para armazenar os DataFrames
dataframes = []

# Loop através dos arquivos CSV e ler cada um
for arquivo in arquivos_csv:
    caminho_completo = os.path.join(diretorio, arquivo)
    df = pd.read_csv(caminho_completo)
    dataframes.append(df)

# Concatenar todos os DataFrames em um único DataFrame
treino = pd.concat(dataframes, ignore_index=True)

In [4]:
# Criar uma lista para armazenar os dados processados para todos os usuários
processed_data = []

# Iterar sobre cada usuário no DataFrame original
for _, row in df.iterrows():
    history = row["history"].split(", ") if pd.notna(row["history"]) else []
    clicks = list(map(int, row["numberOfClicksHistory"].split(", "))) if pd.notna(row["numberOfClicksHistory"]) else []
    time_on_page = list(map(int, row["timeOnPageHistory"].split(", "))) if pd.notna(row["timeOnPageHistory"]) else []
    scroll_percentage = list(map(float, row["scrollPercentageHistory"].split(", "))) if pd.notna(row["scrollPercentageHistory"]) else []

    # Criar um DataFrame temporário com os dados históricos do usuário
    history_df = pd.DataFrame({
        "history": history[:len(clicks)],  # Garantindo que os tamanhos batam
        "clicks": clicks,
        "page_time": time_on_page,
        "scroll_percentage": scroll_percentage
    })

    # Criar a métrica de engajamento para cada página
    history_df["engajamento"] = (history_df["clicks"] * 0.4) + (history_df["page_time"] * 0.4) + (history_df["scroll_percentage"] * 0.2)

    # Ordenar pelo maior engajamento e selecionar os top 5
    top_5_history = history_df.sort_values(by="engajamento", ascending=False).head(5)

    # Criar a estrutura de features
    features = {
        "top_5_history": top_5_history.to_dict(orient="records")
    }

    # Adicionar ao dataset processado
    processed_data.append({
        "userId": row["userId"],
        "features": features
    })

# Criar um novo DataFrame com todos os usuários processados
treino_feature = pd.DataFrame(processed_data)

In [5]:
treino_feature.to_excel("treino_feature.xlsx", index=False)

In [5]:
# Criar diretório para armazenar os modelos
os.makedirs("model", exist_ok=True)

# Carregar os dados
df = pd.read_excel("treino_feature.xlsx")

# Converter a coluna 'features' de string para dicionário
df["features"] = df["features"].apply(literal_eval)

# Extrair métricas relevantes para o modelo
feature_data = []
history_mapping = {}
for _, row in df.iterrows():
    user_id = row["userId"]
    top_5_history = row["features"].get("top_5_history", [])
    
    clicks = [h.get("clicks", 0) for h in top_5_history]
    page_time = [h.get("page_time", 0) for h in top_5_history]
    scroll_percentual = [h.get("scroll_percentual", h.get("scrollPercentage", 0)) for h in top_5_history]
    history = [h.get("history", "") for h in top_5_history]
    
    feature_data.append([
        user_id,
        np.mean(clicks), np.max(clicks), np.sum(clicks),
        np.mean(page_time), np.max(page_time), np.sum(page_time),
        np.mean(scroll_percentual), np.max(scroll_percentual),
        len(history)  # Quantidade de interações como proxy para cold-start
    ])
    
    history_mapping[user_id] = history  # Guardar o histórico de cada usuário

# Criar DataFrame de features
columns = ["userId", "mean_clicks", "max_clicks", "sum_clicks",
           "mean_page_time", "max_page_time", "sum_page_time",
           "mean_scroll", "max_scroll", "history_size"]
features_df = pd.DataFrame(feature_data, columns=columns)

# Separar userId para referência futura
user_ids = features_df["userId"]
X = features_df.drop(columns=["userId"])

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, user_ids, test_size=0.2, random_state=42)

# Treinar o modelo KNN
knn = KNeighborsClassifier(n_neighbors=6, metric='euclidean')
knn.fit(X_train, y_train)

# Treinar o modelo KMeans para cold-start
kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
kmeans.fit(X_scaled)

# Salvar os modelos treinados
with open("model/knn_model.pkl", "wb") as knn_file:
    pickle.dump(knn, knn_file, protocol=pickle.HIGHEST_PROTOCOL)

with open("model/kmeans_model.pkl", "wb") as kmeans_file:
    pickle.dump(kmeans, kmeans_file, protocol=pickle.HIGHEST_PROTOCOL)

with open("model/scaler.pkl", "wb") as scaler_file:
    pickle.dump(scaler, scaler_file, protocol=pickle.HIGHEST_PROTOCOL)

# Salvar history_mapping em formato JSON para facilitar leitura pela API
with open("model/history_mapping.json", "w", encoding="utf-8") as history_file:
    json.dump(history_mapping, history_file, ensure_ascii=False, indent=4)

# Função para recomendar páginas baseado nos 6 perfis mais similares
def recomendar_paginas(user_id):
    if user_id not in history_mapping or len(history_mapping[user_id]) == 0:
        # Cold-start para usuário novo: recomenda páginas com base no cluster do KMeans
        user_index = user_ids[user_ids == user_id].index[0]
        user_features = X_scaled[user_index].reshape(1, -1)
        user_cluster = kmeans.predict(user_features)[0]
        
        # Encontrar usuários no mesmo cluster
        cluster_members = user_ids[kmeans.labels_ == user_cluster]
        
        recomendacoes = set()
        for member_id in cluster_members:
            recomendacoes.update(history_mapping.get(member_id, []))
        
        return list(recomendacoes)[:6]  # Retorna as top 6 páginas recomendadas dos usuários do mesmo cluster
    
    user_index = user_ids[user_ids == user_id].index[0]
    user_features = X_scaled[user_index].reshape(1, -1)
    neighbors = knn.kneighbors(user_features, n_neighbors=6, return_distance=False)[0]
    
    recomendacoes = set()
    for neighbor in neighbors:
        neighbor_id = user_ids.iloc[neighbor]
        recomendacoes.update(history_mapping.get(neighbor_id, []))
    
    return list(recomendacoes)[:6]  # Retorna as top 6 páginas recomendadas dos usuários mais similares

# Teste com um userId específico
#sample_user_id = user_ids.iloc[0]
#recomendacoes = recomendar_paginas(sample_user_id)
#print(f"Recomendações para o usuário {sample_user_id}: {recomendacoes}")


Recomendações para o usuário e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d5092adc513f516b377: ['6767b1cc-fffc-4eb1-8d66-e0f616a8c9e8', 'aa6ca24c-053f-4cb0-9bdc-05d8b6def58a', 'f59aebea-a859-401c-a593-74684984f18c', '12db52fe-fbbf-4e0c-be10-76ab2f945646', '01bc9330-2841-46e0-a13b-a1fad9a66148', 'b3c931c0-ed8d-4b99-8e97-4dba7a564cd0']


In [8]:
#new_user_id = "760e33b6b680b88dea00dea6730ee654e3038075685f9a811f0a7ce910000000"  ### ID inventado 
#recomendacoes = recomendar_paginas(new_user_id)
#print(f"Recomendações para novo usuário: {recomendacoes}")


##077926ded777e56a32eaa708180b1497c120f64a01c07b207d2f57fa75b550f6
##946b216d2fdd2f6ef42c4cbe94b04edfa0d1331ce0507943cc0b8b75e2008776
##aa510684908d1ae0c86193bba5cde7280e52006c65938c6308390c0709cb5ff3

Recomendações para novo usuário: ['d2593c3d-2347-40d9-948c-b6065e8459a9', 'bf257382-74fb-4392-ad6a-143240e39f81', 'f6b5d170-48b9-4f8e-88d4-c84b6668f3bd', 'a36c98b5-f159-48f8-9f5a-1fc6ea9956c8', '89fa73f0-4341-4de4-bb2a-e429ef96bd43', '458bf0ec-efb4-4bfd-9446-c80295e6aa87']
